<a href="https://colab.research.google.com/github/Vedernikov1/ML-DL.models/blob/main/DomAd.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Domain shift
Целью данного блокнота является проверить данные train и test выборки на доменное смещение, то есть понять, отличаются ли у нас даннные на которых мы обучаемся и данные, на которых нашей моделе придется работать

Чтобы понять это, мы для начала получим эмбеддинги предложений из двух наборов данных, а потом обучим классификатор, который должен будет определять принадлежность к тому или другому набору данных. Если классификатор будет обладать низкой точностью (В идеале, в районе 50%, значит между данными нет смещения)

In [ ]:
pip install transformers

In [2]:
import torch
import pandas as pd
import numpy as np

# Библиотека с предобученными трансформерами
from transformers import AutoTokenizer, AutoModel

# Для разбиения train выборки
from sklearn.model_selection import train_test_split

# Библиотеки для классификации
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb

# Для подсчета метрик
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [3]:
# Константы

RANDOM_SEED = 42
TRAIN_FILENAME = 'train.tsv'
TEST_FILENAME = 'test.tsv'
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### Создание датасета для классификации

In [4]:
def read_dataset(filename):
  df = pd.read_csv(filename, sep="\t")
  return np.array(df['title'])

In [8]:
X_train = read_dataset(TRAIN_FILENAME)[:1000] # len = 1000 Чтобы сделать классы сбалансированными
X_test = read_dataset(TEST_FILENAME) # len = 1000

y_train = np.ones(len(X_train)) # ones - первый класс
y_test = np.zeros(len(X_test)) # zeros - второй класс

X_data = np.hstack([X_train, X_test])
y_data = np.hstack([y_train, y_test])

In [9]:
# Разделим полученный датасет на train и test и перемешаем
X_class_train, X_class_test, y_class_train, y_class_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42, shuffle=True)

### Токенизация данных и получение эмбеддингов

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/sbert_large_nlu_ru")

tokenized_train = tokenizer(list(X_class_train), padding=True, truncation=True, max_length=24, return_tensors='pt')
tokenized_train = tokenized_train.to(DEVICE)

tokenized_test = tokenizer(list(X_class_test), padding=True, truncation=True, max_length=24, return_tensors='pt')
tokenized_test = tokenized_test.to(DEVICE)

In [ ]:
# Скачивание предобученной модели
model_RuBERT = AutoModel.from_pretrained("sberbank-ai/sbert_large_nlu_ru").to(DEVICE)

In [12]:
''' Данная функция взята с https://huggingface.co '''


#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [13]:
with torch.no_grad():
    model_output_train = model_RuBERT(**tokenized_train)
with torch.no_grad():
    model_output_test = model_RuBERT(**tokenized_test)

train_embeddings = mean_pooling(model_output_train, tokenized_train['attention_mask'])
test_embeddings = mean_pooling(model_output_test, tokenized_test['attention_mask'])

### Обучение классификационных моделей

In [ ]:
classification_models = dict()

clf = make_pipeline(StandardScaler(), SVC(kernel='rbf', gamma='scale'))
clf.fit(train_embeddings.cpu(), y_class_train)
classification_models['SVM'] = clf

clf = make_pipeline(StandardScaler(), RandomForestClassifier())
clf.fit(train_embeddings.cpu(), y_class_train)
classification_models['RandomForestClassifier'] = clf

clf = make_pipeline(StandardScaler(), LogisticRegression())
clf.fit(train_embeddings.cpu(), y_class_train)
classification_models['LogisticRegeression'] = clf

clf = make_pipeline(StandardScaler(), xgb.XGBClassifier())
clf.fit(train_embeddings.cpu(), y_class_train)
classification_models['GBM'] = clf

### Подсчет метрик

In [16]:
def compute_metrics(y_true, y_pred):
  metric_scores = dict()

  metric_scores['f1_score'] = f1_score(y_true, y_pred)
  metric_scores['roc-auc'] = roc_auc_score(y_true, y_pred)
  metric_scores['accuracy'] = accuracy_score(y_true, y_pred)

  return metric_scores

In [17]:
for model_name, class_model in classification_models.items():
  y_pred = np.array(class_model.predict(test_embeddings.cpu()))
  metrics = compute_metrics(y_class_test, y_pred)

  print(f'{model_name}:')
  for score_name, score in metrics.items():
    print(f'{score_name}: {score}')
  print()

SVM:
f1_score: 0.4533333333333333
roc-auc: 0.48719967999199976
accuracy: 0.4875

RandomForestClassifier:
f1_score: 0.47120418848167533
roc-auc: 0.4947873696842421
accuracy: 0.495

LogisticRegeression:
f1_score: 0.4421052631578947
roc-auc: 0.4697617440436011
accuracy: 0.47

GBM:
f1_score: 0.529262086513995
roc-auc: 0.5374259356483913
accuracy: 0.5375



### Анализ и интерпретируемость

Мы получили качество всех классификаторов в районе 50%, а это в свою очередь означает отсутствие доменного смещения и можно обучать основную модель без доменной адаптации